In [1]:
import django

In [3]:
print django.get_version()

1.10.6


In [6]:
!pwd

/home/jw/Documents/[PYTHON]/acorn수업/0313class_day14_django


# db설정부터 해야한다  
# mysites/settings.py에서 내용 추가
### @@이름@@ 은 변경 필요함

# polls(application)

In [ ]:
from django.shortcuts import render

# Create your views here.

from django.shortcuts import render
from django.shortcuts import get_object_or_404

from django.http import HttpResponseRedirect
from django.core.urlresolvers import reverse
from django.views.generic import ListView, DetailView

# Create your views here.

from polls.models import Choice, Question
import logging
logger = logging.getLogger(__name__)

class IndexView(ListView):
    template_name = 'polls/index.html'
    context_object_name = 'latest_question_list'

    def get_queryset(self):
        """Return the last five published questions."""
        return Question.objects.order_by('-pub_date')[:5]

class DetailView(DetailView):
    model = Question
    template_name = 'polls/detail.html'

class ResultsView(DetailView):
    model = Question
    template_name = 'polls/results.html'

def vote(request, question_id):
    logger.debug('vote().question_id:%s' % question_id)
    p = get_object_or_404(Question, pk=question_id)
    try:
        selected_choice = p.choice_set.get(pk=request.POST['choice'])
    except (KeyError, Choice.DoesNotExist):
        # 투표 폼을 다시 보여주기.
        return render_to_response('polls/detail.html', {
            'poll': p,
            'error_message': "You didn't select a choice.",
        }, context_instance=RequestContext(request))
    else:
        selected_choice.votes += 1
        selected_choice.save()
        # 전송된 POST 자료를 성공적으로 처리한 뒤에는 항상 HttpResponseRedirect를 반환하라.
        # with POST data. This prevents data from being posted twice if a
        # 자료가 두 번 전송되는 경우를 막아준다.
        return HttpResponseRedirect(reverse('polls.views.results', args=(p.id,)))

In [ ]:
from __future__ import unicode_literals

from django.db import models

# Create your models here.

import datetime
from django.utils import timezone
from django.db import models

class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField('date published')

    def __str__(self):
        return self.question_text
    def was_published_recently(self):
        return self.pub_date >= timezone.now() - datetime.timedelta(days=1)

class Choice(models.Model):
    question = models.ForeignKey(Question)
    choice_text = models.CharField(max_length=200)
    votes = models.IntegerField(default=0)

    def __unicode__(self):
        return self.choice_text